In [1]:
import pandas as pd
import os
import numpy as np
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Load data

In [14]:
data = pd.read_csv(os.path.join('..', 'data', 'Devices_Specs.csv'))

def inspect_data(data):
    dataCols = data.columns
    NullSum = data.isnull().sum()
    NullPercent = NullSum/len(data)*100
    UniqueSum = data.nunique()
    ColType = data.dtypes
    # nsamples = 5
    # Samples = [', '.join([str(data[col][i]) for i in range(nsamples)]) for col in data.columns]
    n_UniqueSample = 5
    UniqueSamples = [', '.join([str(data[col].unique()[i]) for i in range(n_UniqueSample if len(
        data[col].unique()) > n_UniqueSample else len(data[col].unique()))]) for col in data.columns]

    Inspec = pd.DataFrame({'Column': dataCols, 'NullSum': NullSum, 'NullPercent%': NullPercent,
                          'UniqueSum': UniqueSum, 'Type': ColType, 'Unique Sample': UniqueSamples})
    Inspec.index = [i for i in range(1, len(data.columns)+1)]
    return Inspec

# Lay cac cot thuoc MEMORY
memoryCols = [col for col in data.columns if re.search('MEMORY', col)]

# inspect data cac cot thuoc DISPLAY
memoryData = data[memoryCols]
inspect_data(memoryData)

,Column,NullSum,NullPercent%,UniqueSum,Type,Unique Sample
1,MEMORY_Card_slot,0,0.000000,158,object,"microSDXC (dedicated slot), microSDXC (uses shared SIM slot), microSDXC, microSDHC (dedicated slot), microSDHC (uses shared SIM slot)"
2,MEMORY_Internal,1792,14.321106,1136,object,"32GB 4GB RAM, 16GB 2GB RAM, 32GB 2GB RAM, 32GB 3GB RAM, 8GB 1GB RAM, 16GB 2GB RAM, 32GB 2GB RAM, 64GB 2GB RAM"


# Card_slot Col

Không có null

In [15]:
print(len(memoryData['MEMORY_Card_slot'].unique()))
print(memoryData['MEMORY_Card_slot'].value_counts())

158
No                                                                                                      3143
microSDHC (dedicated slot)                                                                              2968
microSDXC (dedicated slot)                                                                              2001
microSDXC (uses shared SIM slot)                                                                         979
microSDXC                                                                                                853
microSD (dedicated slot)                                                                                 815
microSDHC                                                                                                702
microSDHC (dedicated slot), 2 GB included                                                                166
miniSD                                                                                                   108
NM (Nano Memory

In [19]:
def  unifyCardSlot(data: pd.DataFrame):
    data['MEMORY_Card_slot'] = data['MEMORY_Card_slot'].str.lower()
    for i in range(len(data['MEMORY_Card_slot'])):
        if 'microsdhc' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'microsdhc'
        
        elif 'microsdxc' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'microsdxc'
        
        elif 'microsd' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'microsd'
            
        elif 'mmc' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'mmc'
        
        elif 'minisd' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'minisd'

        elif 'memory stick micro' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'memory stick micro'

        elif 'memory stick duo' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'memory stick duo'

        elif 'nm (nano memory)' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'nano memory'

        elif 'sdio' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'sdio'
        
        elif 'sd' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'sd'

        elif 'no' in data['MEMORY_Card_slot'][i]:
            data.loc[i, 'MEMORY_Card_slot'] = 'no'
        
        else:
            data.loc[i, 'MEMORY_Card_slot'] = 'unspecified'

In [20]:
memoryDataTemp = memoryData.copy()
unifyCardSlot(memoryDataTemp)
print(memoryDataTemp['MEMORY_Card_slot'].nunique())
memoryDataTemp['MEMORY_Card_slot'].value_counts()

12


microsdhc             4038
microsdxc             3898
no                    3143
microsd                899
minisd                 157
mmc                    121
nano memory             78
memory stick micro      68
unspecified             52
sd                      29
memory stick duo        20
sdio                    10
Name: MEMORY_Card_slot, dtype: int64

# Combine

In [ ]:
def extract_memory_cardslot(data: pd.DataFrame, inplace=False):
    if inplace == False:
        data = data.copy()

    unifyCardSlot(data)

    return data

# Internal Col

In [22]:
print(memoryData['MEMORY_Internal'].isnull().sum())
print(memoryData['MEMORY_Internal'].isnull().sum()/len(memoryData)*100)

1792
14.321106049708304


Tạm sẽ điền null sau

In [24]:
print(memoryData['MEMORY_Internal'].nunique())
print(memoryData['MEMORY_Internal'].value_counts())

1136
8GB 1GB RAM                                                                                                                        788
4GB 512MB RAM                                                                                                                      535
16GB 2GB RAM                                                                                                                       491
32GB 3GB RAM                                                                                                                       307
16GB 1GB RAM                                                                                                                       300
64GB 4GB RAM                                                                                                                       295
4GB 1GB RAM                                                                                                                        282
32GB 2GB RAM                                      

Tách ra làm 2 cột rom và ram, một điện thoại có thể nhiều spec rom ram khác nhau, chọn spec tốt nhất